In [33]:
from tensorflow.keras import layers, models, losses, Model
import numpy as np
import tensorflow as tf

In [56]:
import shutil
import os
# Paths
#test_augmentation
original_dir = "/home/mahdi/logo_detection_model/autoencoder/detected_logos/"
train_dir = "/home/mahdi/logo_detection_model/autoencoder/train"
val_dir = "/home/mahdi/logo_detection_model/autoencoder/validation"

# Split data into train and validation sets
for class_name in os.listdir(original_dir):
    class_path = os.path.join(original_dir, class_name)
    train_class_path = os.path.join(train_dir, class_name)
    val_class_path = os.path.join(val_dir, class_name)
    
    os.makedirs(train_class_path, exist_ok=True)
    os.makedirs(val_class_path, exist_ok=True)

    images = os.listdir(class_path)
    split_idx = int(len(images) * 0.8)  # 80% for training, 20% for validation

    for img_name in images[:split_idx]:
        shutil.copy(os.path.join(class_path, img_name), train_class_path)
    for img_name in images[split_idx:]:
        shutil.copy(os.path.join(class_path, img_name), val_class_path)

In [57]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = "/home/mahdi/logo_detection_model/autoencoder/train"
val_dir = "/home/mahdi/logo_detection_model/autoencoder/validation"
dir = "/home/mahdi/logo_detection_model/autoencoder/detected_logos/"


def safe_preprocess(img):
    try:
        return img  # Pass through valid images
    except Exception as e:
        print("Skipping corrupted image.")
        return np.zeros((224, 224, 3))  # Placeholder for invalid images


# Training Data Generator
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,  
    
    preprocessing_function=safe_preprocess
)

# Validation Data Generator
val_datagen = ImageDataGenerator(
    rescale=1.0 / 255,  
    
    preprocessing_function=safe_preprocess
)

# Create Generators
train_generator = train_datagen.flow_from_directory(
    train_dir,  # Path to training data
    target_size=(224, 224),  # Resize images
    batch_size=16,  # Batch size
    class_mode='input'  # <-- This is important for autoencoders
)

val_generator = val_datagen.flow_from_directory(
     val_dir,  # Path to validation data
    target_size=(224, 224),  # Resize images
    batch_size=16,  # Batch size
    class_mode='input'  # <-- This is important for autoencoders
)

print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {val_generator.samples}")

Found 206 images belonging to 1 classes.
Found 52 images belonging to 1 classes.
Training samples: 206
Validation samples: 52


#### Using class_mode = 'input' for autoencoder

In [58]:
print(f"Train generator samples: {train_generator.samples}")
print(f"Validation generator samples: {val_generator.samples}")


Train generator samples: 206
Validation generator samples: 52


In [59]:
# Try fetching a batch of data
batch = next(iter(train_generator))
print("Train batch shapes:", [x.shape for x in batch])


Train batch shapes: [(16, 224, 224, 3), (16, 224, 224, 3)]


In [60]:
# Try fetching a batch of data
batch = next(iter(val_generator))
print("val batch shapes:", [x.shape for x in batch])

val batch shapes: [(16, 224, 224, 3), (16, 224, 224, 3)]


In [61]:
# batch = next(iter(val_generator))
# print(f"Input batch shape: {batch.shape}")  # Should match (batch_size, 224, 224, 3)


In [62]:
len(train_generator)

13

In [63]:
len(val_generator)

4

In [83]:

# # Autoencoder architecture
# def build_autoencoder(input_shape=(224, 224, 3), latent_dim=128):
#     # Encoder
#     input_img = layers.Input(shape=input_shape)
#     x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(input_img)
#     x = layers.MaxPooling2D((2, 2), padding="same")(x)
#     x = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x)
#     x = layers.MaxPooling2D((2, 2), padding="same")(x)
#     x = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(x)
#     x = layers.Flatten()(x)
#     latent = layers.Dense(latent_dim, activation="relu")(x)  # Latent space

#     # Decoder
#     x = layers.Dense(28 * 28 * 128, activation="relu")(latent)
#     x = layers.Reshape((28, 28, 128))(x)
#     x = layers.Conv2DTranspose(64, (3, 3), activation="relu", padding="same")(x)
#     x = layers.UpSampling2D((2, 2))(x)
#     x = layers.Conv2DTranspose(32, (3, 3), activation="relu", padding="same")(x)
#     x = layers.UpSampling2D((2, 2))(x)
#     output_img = layers.Conv2DTranspose(3, (3, 3), activation="sigmoid", padding="same")(x)

#     # Autoencoder Model
#     autoencoder = models.Model(input_img, output_img)
#     encoder = models.Model(input_img, latent)  # Separate encoder model for feature extraction

#     return autoencoder, encoder



In [64]:
from tensorflow.keras import layers, models

# Encoder
encoder_input = layers.Input(shape=(224, 224, 3))
x = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(encoder_input)
x = layers.MaxPooling2D((2, 2), padding="same")(x)  # Down to (112, 112, 64)
x = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(x)
x = layers.MaxPooling2D((2, 2), padding="same")(x)  # Down to (56, 56, 128)
encoder_output = x

# Decoder
x = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(encoder_output)
x = layers.UpSampling2D((2, 2))(x)  # Up to (112, 112, 128)
x = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x)
x = layers.UpSampling2D((2, 2))(x)  # Up to (224, 224, 64)
decoder_output = layers.Conv2D(3, (3, 3), activation="sigmoid", padding="same")(x)  # Match input shape

# Autoencoder Model
# decoder_output = layers.Conv2D(3, (3, 3), activation="sigmoid", padding="same")(decoder_output)
# autoencoder = Model(encoder_input, decoder_output)
autoencoder = models.Model(encoder_input, decoder_output)
autoencoder.compile(optimizer="adam", loss="mse")

autoencoder.summary()


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_27 (Conv2D)              │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_28 (Conv2D)              │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_29 (Conv2D)              │ (None, 56, 56, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_10 (UpSampling2D) │ (None, 112, 112, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_30 (Conv2D)              │ (None, 112, 112, 64)   │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_11 (UpSampling2D) │ (None, 224, 224, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_31 (Conv2D)              │ (None, 224, 224, 3)    │         1,731 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 298,755 (1.14 MB)

 Trainable params: 298,755 (1.14 MB)

 Non-trainable params: 0 (0.00 B)

In [114]:
# # Build and compile the autoencoder
# autoencoder, encoder = build_autoencoder()
# autoencoder.compile(optimizer="adam", loss=losses.MeanSquaredError())

In [65]:
print("Autoencoder input shape:", autoencoder.input_shape)
print("Autoencoder output shape:", autoencoder.output_shape)


Autoencoder input shape: (None, 224, 224, 3)
Autoencoder output shape: (None, 224, 224, 3)


In [66]:
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = val_generator.samples // val_generator.batch_size
print(f"Steps per epoch: {steps_per_epoch}, Validation steps: {validation_steps}")


Steps per epoch: 12, Validation steps: 3


In [10]:
# Check the generators
assert train_generator.samples > 0, "Training directory is empty or misconfigured!"
assert val_generator.samples > 0, "Validation directory is empty or misconfigured!"

steps_per_epoch = max(1, train_generator.samples // train_generator.batch_size)
validation_steps = max(1, val_generator.samples // val_generator.batch_size)


## Test to debug Error: None values not supported.

In [36]:
# Debug the data generators
batch = next(iter(train_generator))
print(f"Batch shape: {batch.shape if batch is not None else 'None'}")


Batch shape: (13, 224, 224, 3)


In [37]:
steps_per_epoch = max(1, train_generator.samples // train_generator.batch_size)
validation_steps = max(1, val_generator.samples // val_generator.batch_size)

print(f"Steps per epoch: {steps_per_epoch}")
print(f"Validation steps: {validation_steps}")


Steps per epoch: 1
Validation steps: 1


In [39]:
batch = next(iter(train_generator))
print(f"Train batch shape: {batch.shape}")


Train batch shape: (13, 224, 224, 3)


In [40]:
batch = next(iter(val_generator))
print(f"Validation batch shape: {batch.shape}")


Validation batch shape: (7, 224, 224, 3)


/home/mahdi/logo_detection_model/.venv/lib/python3.12/site-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [68]:
# predictions = autoencoder.predict(batch)
# print(f"Prediction shape: {predictions.shape}")


In [43]:
batch = next(iter(train_generator))
predictions = autoencoder(batch)
print(f"Prediction shape: {predictions.shape}")


Prediction shape: (13, 224, 224, 3)


## Run the Model

In [69]:
history = autoencoder.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    steps_per_epoch=max(1, train_generator.samples // train_generator.batch_size),
    validation_steps=max(1, val_generator.samples // val_generator.batch_size),
    verbose=1
)


Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 354ms/step - loss: 0.1265 - val_loss: 0.0484
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0363 - val_loss: 0.0470
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 341ms/step - loss: 0.0470 - val_loss: 0.0437
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0421 - val_loss: 0.0329
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 339ms/step - loss: 0.0338 - val_loss: 0.0218
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.0202 - val_loss: 0.0232
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 337ms/step - loss: 0.0197 - val_loss: 0.0159
Epoch 8/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.0153 - val_loss: 0.0153
Epoch 9/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 342ms/step - loss: 0.0137 - val_loss: 0.0148
Epoch 10/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0147 - val_loss: 0.0141
Epoch 11/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 340ms/step - loss: 0.0114 - val_loss: 0.0125
Epoch 12/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - los

In [38]:
# Train the autoencoder
history = autoencoder.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    batch_size = 4,
    # steps_per_epoch=len(train_generator),
    # validation_steps=len(val_generator)
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=val_generator.samples // val_generator.batch_size

)


Epoch 1/50


ValueError: None values not supported.

In [ ]:
# Save the encoder for feature extraction
encoder.save("logo_encoder.h5")


In [ ]:
# Compare similarity using latent features
def calculate_similarity(encoder, input_logo, valid_logos):
    input_features = encoder.predict(input_logo)
    similarities = []
    for valid_logo in valid_logos:
        valid_features = encoder.predict(valid_logo)
        similarity = np.dot(input_features, valid_features.T) / (
            np.linalg.norm(input_features) * np.linalg.norm(valid_features)
        )
        similarities.append(similarity)
    return similarities



In [ ]:
# Example usage
input_logo = ...  # Load input image and preprocess
valid_logos = [...]  # List of preprocessed valid logos
similarities = calculate_similarity(encoder, input_logo, valid_logos)
